# 第五章：格式化输出和为模型代言

- [课程内容](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:


# 导入 Python 内置的正则表达式库
import re
from openai import OpenAI

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME
%store -r BASE_URL

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# 为预填充文本添加新参数，默认值为空字符串
def get_completion(prompt: str, system_prompt="", prefill=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    return response.choices[0].message.content

---

## 课程内容

**模型可以通过各种方式格式化其输出**。您只需要要求它这样做！

其中一种方式是使用 XML 标签将响应与任何其他多余文本分离。您已经学会了可以使用 XML 标签使您的提示对模型更清晰、更易解析。事实证明，您还可以要求模型**使用 XML 标签使其输出对人类更清晰、更易理解**。

### 示例

还记得我们在第二章中通过要求模型完全跳过前言来解决的"诗歌前言问题"吗？事实证明，我们也可以通过**告诉模型将诗歌放在 XML 标签中**来实现类似的结果。

In [ ]:
# 可变内容
ANIMAL = "兔子"

# 包含可变内容占位符的提示模板
PROMPT = f"请写一首关于{ANIMAL}的俳句。将其放在 <haiku> 标签中。"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

为什么我们要这样做？嗯，将输出放在 **XML 标签中允许最终用户通过编写简短程序来提取 XML 标签之间的内容，从而可靠地获得诗歌且仅获得诗歌**。

这种技术的扩展是**将第一个 XML 标签放在 `assistant` 轮次中**。当您在 `assistant` 轮次中放置文本时，您基本上是在告诉模型它已经说了些什么，并且应该从那个点继续。这种技术被称为"为模型代言"或"预填充模型的响应"。

下面，我们用第一个 `<haiku>` XML 标签做了这件事。注意模型如何直接从我们停下的地方继续。

In [ ]:
# 可变内容
ANIMAL = "猫"

# 包含可变内容占位符的提示模板
PROMPT = f"请写一首关于{ANIMAL}的俳句。将其放在 <haiku> 标签中。"

# 模型响应的预填充
PREFILL = "<haiku>"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户轮次:")
print(PROMPT)
print("\n助手轮次:")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

模型还擅长使用其他输出格式样式，特别是 `JSON`。如果您想强制执行 JSON 输出（不是确定性的，但接近），您也可以用开括号 `{` 预填充模型的响应。

In [ ]:
# 可变内容
ANIMAL = "猫"

# 包含可变内容占位符的提示模板
PROMPT = f"请写一首关于{ANIMAL}的俳句。使用 JSON 格式，键为 \"first_line\"、\"second_line\" 和 \"third_line\"。"

# 模型响应的预填充
PREFILL = "{"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

下面是一个**在同一提示中使用多个输入变量和输出格式规范的示例，全部使用 XML 标签完成**。

In [ ]:
# 第一个输入变量
EMAIL = "嗨，扎克，只是想快速了解一下你应该写的那个提示的进展情况。"

# 第二个输入变量
ADJECTIVE = "古英语风格"

# 包含可变内容占位符的提示模板
PROMPT = f"嗨，AI助手。这里有一封邮件：<email>{EMAIL}</email>。让这封邮件更具{ADJECTIVE}。将新版本写在 <{ADJECTIVE}_email> XML 标签中。"

# 模型响应的预填充（现在是带有变量的 f-string）
PREFILL = f"<{ADJECTIVE}_email>"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### 额外课程

如果您通过 API 调用模型，您可以将结束 XML 标签传递给 `stop_sequences` 参数，以便模型在发出您所需的标签后停止采样。这可以通过消除模型在已经给出您关心的答案后的结束语来节省金钱和最后令牌的时间。

如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 5.1 - 史蒂芬·库里 GOAT](#exercise-51---steph-curry-goat)
- [练习 5.2 - 两首俳句](#exercise-52---two-haikus)
- [练习 5.3 - 两首俳句，两种动物](#exercise-53---two-haikus-two-animals)

### 练习 5.1 - 史蒂芬·库里 GOAT
被迫做出选择时，模型通常指定迈克尔·乔丹为史上最佳篮球运动员。我们能让模型选择其他人吗？

更改 `PREFILL` 变量以**强制模型做出详细论证，证明史蒂芬·库里是史上最佳篮球运动员**。尽量不要更改除 `PREFILL` 之外的任何内容，因为这是本练习的重点。

In [ ]:
# 包含可变内容占位符的提示模板
PROMPT = f"谁是史上最伟大的篮球运动员？请选择一个具体的球员。"

# 模型响应的预填充
PREFILL = ""

# 获取模型的响应
response = get_completion(PROMPT, prefill=PREFILL)

# 用于评分练习正确性的函数
def grade_exercise(text):
    return bool(re.search("勇士", text) or re.search("Warrior", text) or re.search("库里", text) or re.search("Curry", text))

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("这个练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_5_1_hint; print(exercise_5_1_hint)

### 练习 5.2 - 两首俳句
使用 XML 标签修改下面的 `PROMPT`，使模型写关于动物的两首俳句而不是一首。应该清楚地看出一首诗在哪里结束，另一首在哪里开始。

In [ ]:
# 可变内容
ANIMAL = "猫"

# 包含可变内容占位符的提示模板
PROMPT = f"请写关于{ANIMAL}的俳句。将其放在 <haiku> 标签中。"

# 模型响应的预填充
PREFILL = "<haiku>"

# 获取模型的响应
response = get_completion(PROMPT, prefill=PREFILL)

# 用于评分练习正确性的函数
def grade_exercise(text):
    return bool(
        (re.search("猫", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("这个练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_5_2_hint; print(exercise_5_2_hint)

### 练习 5.3 - 两首俳句，两种动物
修改下面的 `PROMPT`，使**模型产生关于两种不同动物的两首俳句**。使用 `{ANIMAL1}` 作为第一个替换的占位符，`{ANIMAL2}` 作为第二个替换的占位符。

In [ ]:
# 第一个输入变量
ANIMAL1 = "猫"

# 第二个输入变量
ANIMAL2 = "狗"

# 包含可变内容占位符的提示模板
PROMPT = f"请写一首关于{ANIMAL1}的俳句。将其放在 <haiku> 标签中。"

# 获取模型的响应
response = get_completion(PROMPT)

# 用于评分练习正确性的函数
def grade_exercise(text):
    return bool(re.search("尾巴", text.lower()) and re.search("猫", text.lower()) and re.search("<haiku>", text))

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("这个练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_5_3_hint; print(exercise_5_3_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，您就可以进入下一章了。愉快的提示工程学习！

---

## 示例练习场

这是一个供您自由试验本课程中展示的提示示例并调整提示以查看如何影响模型响应的区域。

In [ ]:
# 可变内容
ANIMAL = "兔子"

# 包含可变内容占位符的提示模板
PROMPT = f"请写一首关于{ANIMAL}的俳句。将其放在 <haiku> 标签中。"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容
ANIMAL = "猫"

# 包含可变内容占位符的提示模板
PROMPT = f"请写一首关于{ANIMAL}的俳句。将其放在 <haiku> 标签中。"

# 模型响应的预填充
PREFILL = "<haiku>"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户轮次:")
print(PROMPT)
print("\n助手轮次:")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# 可变内容
ANIMAL = "猫"

# 包含可变内容占位符的提示模板
PROMPT = f"请写一首关于{ANIMAL}的俳句。使用 JSON 格式，键为 \"first_line\"、\"second_line\" 和 \"third_line\"。"

# 模型响应的预填充
PREFILL = "{"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# 第一个输入变量
EMAIL = "嗨，扎克，只是想快速了解一下你应该写的那个提示的进展情况。"

# 第二个输入变量
ADJECTIVE = "古英语风格"

# 包含可变内容占位符的提示模板
PROMPT = f"嗨，AI助手。这里有一封邮件：<email>{EMAIL}</email>。让这封邮件更具{ADJECTIVE}。将新版本写在 <{ADJECTIVE}_email> XML 标签中。"

# 模型响应的预填充（现在是带有变量的 f-string）
PREFILL = f"<{ADJECTIVE}_email>"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户轮次")
print(PROMPT)
print("\n助手轮次")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))